# Optimizing number of frames

This is coming from the question ([issue 376](https://github.com/pySTEPS/pysteps/issues/376)) on how to optimize the Optical Flow (OF) motion vectors in pySTEPS, and it is taking advantage of *ray* for computing this optimization. Other parameters of the OF computation could also be optimized but in this example just the number of frames is tested.

**Caution:** This notebook is an example, but a larger dataset should be used to optimize the number of frames so that the results are statistically significant and sound.


## Set up the Colab environment

First, let's set up our working environment. Note that these steps are specific for Google Colab. For more information on how to install pysteps see [these instructions](https://pysteps.readthedocs.io/en/latest/user_guide/install_pysteps.html).

### Install dependencies

First, let's install the latest available versions of pysteps and ray using pip. This will also install the minimal dependencies needed to run pysteps.

In [ ]:
!pip install pysteps "ray[tune]" --quiet

### Getting the example data

Now that we have the environment ready, let's install the example data and configure pysteps by following [this tutorial](https://pysteps.readthedocs.io/en/latest/user_guide/example_data.html).

First, we will use the [pysteps.datasets.download_pysteps_data()](https://pysteps.readthedocs.io/en/latest/generated/pysteps.datasets.download_pysteps_data.html) function to download the data.

In [ ]:
# Import the helper functions
from pysteps.datasets import download_pysteps_data, create_default_pystepsrc

# Download the pysteps data in the "pysteps_data"
download_pysteps_data("pysteps_data")

Next, we need to create a default configuration file that points to the downloaded data.
By default, pysteps will place the configuration file in `$HOME/.pysteps` (unix and Mac OS X) or `$USERPROFILE/pysteps` (windows).
To quickly create a configuration file, we will use the [pysteps.datasets.create_default_pystepsrc()](https://pysteps.readthedocs.io/en/latest/generated/pysteps.datasets.create_default_pystepsrc.html#pysteps.datasets.create_default_pystepsrc) helper function.

In [ ]:
# If the configuration file is placed in one of the default locations
# (https://pysteps.readthedocs.io/en/latest/user_guide/set_pystepsrc.html#configuration-file-lookup)
# it will be loaded automatically when pysteps is imported.
config_file_path = create_default_pystepsrc("pysteps_data")

Since pysteps was already initialized in this notebook, we need to load the new configuration file and update the default configuration.

In [ ]:
# Import pysteps and load the new configuration file
import pysteps

_ = pysteps.load_config_file(config_file_path, verbose=True)

### Load the input data

Pysteps comes with few precipitation datasets with precipitation events from different countries and providers. We can see the available datasets with `datasets.info()`:

In [ ]:
from pysteps import datasets
datasets.info()

To import an example dataset, we use the [load_dataset()](https://pysteps.readthedocs.io/en/latest/generated/pysteps.datasets.load_dataset.html) helper function from the `pysteps.datasets` module.

For this tutorial, we will use three events from MeteoSwiss: the first two events for our tuning set, while `mch3` will serve for the validation at the end.

In [ ]:
# import the 2-hour precipitation events
# i.e., a sequence of 24 precipitation frames with 5 minute temporal resolution
mch1 = datasets.load_dataset("mch", frames=24)
mch2 = datasets.load_dataset("mch2", frames=24)
mch3 = datasets.load_dataset("mch3", frames=24)

tune_set = (mch1, mch3)
val_set = (mch2,)

## Set up the experiment
**Evaluate Performance Metrics**

   - Use performance metrics such as Root Mean Squared Error (RMSE), Critical Success Index (CSI), or Heidke Skill Score (HSS) to evaluate the accuracy of nowcasts generated using different numbers of frames.
   - Calculate these metrics for each number of frames across all runs but same forecasting period to avoid dependence on the variability of the observation.

**Analyze Computational Load**
   - Measure the computational time and resources required for processing different numbers of frames.
   - Ensure that the increased accuracy justifies any additional computational costs.

In [ ]:
from pysteps.verification.detcontscores import det_cont_fct
from pysteps.verification.detcatscores import det_cat_fct

def evaluation_fn(fct, obs):
    rmse = det_cont_fct(fct, obs, scores='RMSE',axis=(1,2))
    #csi = det_cat_fct(fct, obs, 0.1, scores='CSI')
    #we could create a normalized multiscore value using several scores
    #but for now we are just returning an array with the lead-time rmse
    return rmse['RMSE']

Here we define the function that will compute a single trial of the optimization experiment. Such function takes a sample of the parameter space as an input and outputs the resulting error.

The trial consists of run of 1 hour at the end of the 2-hours event to account for the variability of the results as a function of the lead-time. The initial hour is the range of number of frames that we are testing. The final error is the average error of all runs.

In [ ]:
from pysteps import motion, nowcasts
from pysteps.utils import transformation

N_LEADTIMES = 12

def pysteps_objective(config, events=None, time_step=None):

    # select nowcasting methods
    estimate_motion = motion.get_method("LK")
    extrapolate = nowcasts.get_method("extrapolation")

    n_frames = config['n_frames']

    error = 0
    n = 0
    for event in events:

        precip_data, metadata, __ = event
        n_total_frames = precip_data.shape[0]

        # log-transform the data to dBR.
        precip_data_dbr, __ = transformation.dB_transform(
            precip_data, metadata, threshold=0.1, zerovalue=-15.0
        )

        # prepare the data for the run
        obs_precip = precip_data[n_total_frames-N_LEADTIMES:n_total_frames]

        input_precip_dbr = precip_data_dbr[n_total_frames-N_LEADTIMES-n_frames:n_total_frames-N_LEADTIMES]
        input_precip = precip_data[n_total_frames-N_LEADTIMES-1]

        # compute the nowcast
        motion_field = estimate_motion(input_precip_dbr)
        forecast_precip = extrapolate(input_precip, motion_field, N_LEADTIMES)

        # evaluate the nowcast
        if time_step is None:
          error += evaluation_fn(forecast_precip, obs_precip)
        else:
          error += evaluation_fn(forecast_precip, obs_precip)[time_step]
        n += 1
        # loop the event

    return {"mean_error": error / n}

Finally, we need to define the [search space](https://docs.ray.io/en/latest/tune/api/search_space.html) for our experiment. This step is critical to the success of the tuning, as it is important to select the most sensitive parameters and to define a reasonable range of values to be explored during the tuning. Typically, one will need to go through multiple iterations before finding a satisfying set up.

Below we define the search space which in this case it is the number of frames. Usually ray is optimal for multiparameter models but for this example just one is used. The number of frames is defined by the input array in [dense Lucas-Kanade](https://pysteps.readthedocs.io/en/stable/generated/pysteps.motion.lucaskanade.dense_lucaskanade.html#pysteps.motion.lucaskanade.dense_lucaskanade) optical flow routine. For each parameter, we need to specify the sampling strategy, for example in this case the min (2 samples) and max (12 samples) limits.

In [ ]:
from ray import tune

#Original random
# param_space = {
#     "n_frames": tune.qrandint(2, 12, 1),
# }

param_space = {
    "n_frames": tune.choice(list(range(2, 13))),
}

## Launch the experiment

We can now launch the experiment. We ask for 12 trials because of this simple experiment. If not specified, ray will use a random search by default, but more sophisticated [search algorithms](https://docs.ray.io/en/latest/tune/api/suggestion.html#tune-search-alg) are available.


In [ ]:
import ray

N_TRIALS = 12

results=[]

#This is a loop in the lead-times range
for i_leadtime in range(12):
  ray.shutdown()
  ray.init(configure_logging=False)

  tuner = tune.Tuner(
      trainable=tune.with_parameters(
          pysteps_objective,
          events=tune_set,
          time_step=i_leadtime,
      ),
      tune_config=tune.TuneConfig(
          metric="mean_error",
          mode="min",
          num_samples=N_TRIALS,
      ),
      param_space=param_space,
  )

  results.append(tuner.fit())

It is important to mention, that in the case of this example, similar results could be obtained without the use of *ray* due to the fact that only one parameter is actually computed for 11 specific values. This could be done as follow:

In [ ]:
list_n_frames = list(range(2, 13))
results_noray=[]
for n_frame in list_n_frames:
  results_noray.append(pysteps_objective({'n_frames':n_frame}, events=tune_set, time_step=None))

## Analyse the results

The first and most simple question we can ask is to return the best parameter set for our problem:

In [ ]:
best_param_set = [result.get_best_result().config for result in results]
best_param_set

## too avoid multiple lead-times optimization, we fix the first lead-time
results = results[0]
best_param_set = best_param_set[0]

We can verify that this results are matching with the ones obtained without using *ray* library. Hoowever, the simplicity of the previous code with ray will change to a slightly more complex code to obtain similar outputs.

In [ ]:
import numpy as np

search_matrix = np.vstack([item['mean_error'] for item in results_noray])

# Lead-time range
lead_time = np.arange(1, 13)

# Optimal n_frames
optimal_n_frames = np.argmin(search_matrix, axis=0) + min(list_n_frames)

# Value of the minimum
optimal_value = np.min(search_matrix, axis=0)

# Print the table with aligned columns
print('{:<12} {}'.format('Lead-time:    ', ' '.join(f'{x:5d}' for x in lead_time)))
print('{:<12} {}'.format('Opt n_frames: ', ' '.join(f'{x:5d}' for x in optimal_n_frames)))
print('{:<12} {}'.format('Optimal value:', ' '.join(f'{x:5.2f}' for x in optimal_value)))

An interesting analysis of the results consists in plotting the error of each trial as a function of its parameter values. Such plots are known as "dot plots". By looking at the shape of the cloud of points, one can get a good idea about the sensitivity of a given parameter. For instance, a flat curve usually indicates weak sensitivity: no matter what parameter value is selected, the resulting error doesn't seem to change much. Conversely, a clear spike in the plot indicates a sensitive parameter.

In [ ]:
import matplotlib.pyplot as plt
from math import ceil

def make_dot_plots(tune_results, yaxis, ylog=True, reference=None):
    tune_df = tune_results.get_dataframe()
    tune_best = tune_results.get_best_result()
    col_params = sorted([col for col in tune_df.columns if "config/" in col])
    ncols = 1
    nrows = ceil(len(col_params) / ncols)
    fig, axs = plt.subplots(nrows, ncols, figsize=(ncols * 3.5, nrows * 3))
    for n, par in enumerate(col_params):
        parsed_par = par.split("/")[1]
        try:
          ax = axs.flat[n]
        except:
          ax = axs
        ax.scatter(tune_df[par], tune_df[yaxis], alpha=0.5)
        ax.scatter(tune_best.config[parsed_par], tune_best.metrics[yaxis], c="r")
        if reference:
            ax.axhline(reference, c="k", ls=":")
        if ylog:
            ax.set_yscale("log")
        ax.set_xlabel(parsed_par)
        if n % ncols == 0:
            ax.set_ylabel(yaxis)
        else:
            ax.axes.get_yaxis().set_visible(False)

    plt.tight_layout()

# compute error for default settings
tune_error_default = pysteps_objective({'n_frames':3}, tune_set)["mean_error"]

make_dot_plots(results, "mean_error", reference=tune_error_default)

The dashed horizontal line represents the skill of the default parameter set (i.e., without tuning), while the red dot represents the best paramter set.

Finally, let's compare the default and best parameter sets on the tune and validation events. This will provide some indication on the generability of the optimized parameter set.

In [ ]:
import pandas as pd

# compute validation scores
val_error_best = pysteps_objective(best_param_set, val_set)["mean_error"]
val_error_default = pysteps_objective({'n_frames':3}, val_set)["mean_error"]

In [ ]:
data = {
    "default": {
        "tune": tune_error_default,
        "val": val_error_default,
    },
    "best": {
        "tune": results.get_best_result().metrics["mean_error"],
        "val": val_error_best,
    },
}

df = pd.DataFrame(data)

# Plotting with adjusted y-axis limits
ax = df.plot.bar()

# Set the y-axis limits to focus on the data range
y_min = df.min().min() - 0.1 * (df.max().max() - df.min().min())
y_max = df.max().max() + 0.1 * (df.max().max() - df.min().min())
ax.set_ylim(y_min, y_max)

And in this case it seems the obtained optimal value is also optimal for the validation event.